In [18]:
import pandas as pd
import numpy as np
import glob
import datetime as dt
import tqdm
import joblib
import pytz
import xarray as xr
import cartopy.crs as ccrs
from sklearn.cluster import KMeans as kmeans
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.ticker import ScalarFormatter, MultipleLocator
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes as inset
from matplotlib.patches import Circle

import sharppy.plot.skew as skew
from sharppy.sharptab import winds, utils, params, thermo, interp, profile
from sharppy.io.spc_decoder import SPCDecoder

from metpy.calc import dewpoint_from_relative_humidity

def plot_sounding(time, lat, lon, plot=False):

    str_fmt = "/g/data/rt52/era5/pressure-levels/reanalysis/%s/%d/%s_era5_oper_pl_%s*.nc" 
    str_fmt_sfc = "/g/data/rt52/era5/single-levels/reanalysis/%s/%d/%s_era5_oper_sfc_%s*.nc"
    
    _, latlon = get_point_data(time,lat,lon,50,["mu_cape"],plot=False,vmin=None,vmax=None)
    lat = latlon[0]
    lon = latlon[1]
    
    z=xr.open_dataset(glob.glob(str_fmt % ("z",time.year,"z",time.strftime("%Y%m01")))[0])["z"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    t=xr.open_dataset(glob.glob(str_fmt % ("t",time.year,"t",time.strftime("%Y%m01")))[0])["t"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    r=xr.open_dataset(glob.glob(str_fmt % ("r",time.year,"r",time.strftime("%Y%m01")))[0])["r"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    u=xr.open_dataset(glob.glob(str_fmt % ("u",time.year,"u",time.strftime("%Y%m01")))[0])["u"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    v=xr.open_dataset(glob.glob(str_fmt % ("v",time.year,"v",time.strftime("%Y%m01")))[0])["v"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")

    ts=xr.open_dataset(glob.glob(str_fmt_sfc % ("2t",time.year,"2t",time.strftime("%Y%m01")))[0])["t2m"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    dps=xr.open_dataset(glob.glob(str_fmt_sfc % ("2d",time.year,"2d",time.strftime("%Y%m01")))[0])["d2m"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    us=xr.open_dataset(glob.glob(str_fmt_sfc % ("10u",time.year,"10u",time.strftime("%Y%m01")))[0])["u10"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    vs=xr.open_dataset(glob.glob(str_fmt_sfc % ("10v",time.year,"10v",time.strftime("%Y%m01")))[0])["v10"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    ps=xr.open_dataset(glob.glob(str_fmt_sfc % ("sp",time.year,"sp",time.strftime("%Y%m01")))[0])["sp"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    zs=xr.open_dataset(glob.glob(str_fmt_sfc % ("z",time.year,"z",time.strftime("%Y%m01")))[0])["z"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")   
    
    p=z.level
    dp=dewpoint_from_relative_humidity(t,r)
    prof_pres = np.flip(p.values.squeeze())
    prof_hgt = np.flip(z.values.squeeze()/9.8)
    prof_tmpc = np.flip(t.values.squeeze()-273.15)
    prof_dwpc = np.flip(np.array(dp).squeeze())
    prof_u = np.flip(u.values.squeeze()*1.94)
    prof_v = np.flip(v.values.squeeze()*1.94)
    agl_inds = prof_pres <= (ps.values/100.)
    prof = profile.create_profile(pres=np.insert(prof_pres[agl_inds], 0, ps.values/100.),
                                  hght=np.insert(prof_hgt[agl_inds], 0, zs.values/9.8),
                                  tmpc=np.insert(prof_tmpc[agl_inds], 0, ts.values-273.15),
                                  dwpc=np.insert(prof_dwpc[agl_inds], 0, dps.values-273.15),
                                  u=np.insert(prof_u[agl_inds], 0, us.values*1.94),
                                  v=np.insert(prof_v[agl_inds], 0, vs.values*1.94))
    sfcpcl = params.parcelx( prof, flag=1 )
    mupcl = params.parcelx( prof, flag=3 )
    mlpcl = params.parcelx( prof, flag=4 )    
    if plot:
        # Bounds of the pressure axis 
        plt.figure()
        pb_plot=1050
        pt_plot=100
        dp_plot=10
        plevs_plot = np.arange(pb_plot,pt_plot-1,-dp_plot)
        # Open up the text file with the data in columns (e.g. the sample OAX file distributed with SHARPpy)
        title = time.replace(minute=0).strftime('%Y%m%d/%H%M') + ' UTC   Latitude: ' + str(lat) + ', Longitude: ' + str(lon) + '   (ERA5)'

        # Set up the figure in matplotlib.
        gs = gridspec.GridSpec(4,6, width_ratios=[1,5,1,1,1,1])
        ax = plt.subplot(gs[0:3, 0:2], projection='skewx')
        skew.draw_title(ax, title)
        ax.grid(True)
        plt.grid(True)

        # Plot the background variables
        presvals = np.arange(1000, 0, -10)
        ax.semilogy(prof.vtmp[~prof.vtmp.mask], prof.pres[~prof.vtmp.mask], 'r', lw=2)
        #ax.semilogy(prof.tmpc[~prof.tmpc.mask], prof.pres[~prof.tmpc.mask], 'r', lw=2)
        ax.semilogy(prof.dwpc[~prof.dwpc.mask], prof.pres[~prof.dwpc.mask], 'g', lw=2)

        # Plot the parcel trace, but this may fail.  If it does so, inform the user.
        ax.semilogy(sfcpcl.ttrace, sfcpcl.ptrace, 'k:')
        ax.semilogy(mlpcl.ttrace, mlpcl.ptrace, color='tab:blue', ls=":")
        ax.semilogy(mupcl.ttrace, mupcl.ptrace, color='tab:red', ls=":")

        # Disables the log-formatting that comes with semilogy
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.set_yticks(np.linspace(100,1000,10))
        ax.set_ylim(1050,100)
        [ax.text(-0.3, p, str(round(h/1000,1)) + " km", transform=ax.get_yaxis_transform(), va="center") 
             for h, p in zip(prof.hght[prof.pres >= 100][3:-1:2], prof.pres[prof.pres >= 100][3:-1:2])]

        # Plot the hodograph data.
        inset_axes = skew.draw_hodo_inset(ax, prof)
        skew.plotHodo(inset_axes, prof.hght, prof.u, prof.v, color='r')

        # Draw the wind barbs axis and everything that comes with it.
        ax.set_xlim(-40,50)
        ax.set_xticks(np.arange(-40,60,10))
        ax2 = plt.subplot(gs[0:3,2])
        skew.plot_wind_axes(ax2)
        skew.plot_wind_barbs(ax2, prof.pres, prof.u, prof.v)
        srwind = params.bunkers_storm_motion(prof)
        gs.update(left=0.05, bottom=0.05, top=0.95, right=1, wspace=0.025)    

        ax3 = plt.subplot(gs[0:3,4:6])
        ax3.semilogy(prof.thetae[(~prof.thetae.mask) & (prof.pres >= 300)], prof.pres[(~prof.thetae.mask) & (prof.pres >= 300)], 'tab:blue', lw=2, ls="--")
        ax3.yaxis.set_major_formatter(ScalarFormatter())
        ax3.set_yticks(np.linspace(100,1000,10))
        ax3.set_ylim(1050,100)        

        sfc = prof.pres[prof.sfc]
        p6km = interp.pres(prof, interp.to_msl(prof, 6000.))
        sfc_6km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p6km)
        s06 = utils.comp2vec(sfc_6km_shear[0], sfc_6km_shear[1])[1] * 0.514444
        print("MUCAPE: ",mupcl.bplus, "MLCAPE: ",mlpcl.bplus, "SBCAPE: ",sfcpcl.bplus, "S06: ",s06, "DCAPE",params.dcape(prof)[0])

    return prof, mupcl
        
def get_point_data(time,lat,lon,r,var,plot=False,vmin=None,vmax=None):
    
    f = xr.open_dataset(glob.glob("/g/data/eg3/ab4502/ExtremeWind/aus/era5/era5_"\
                            +time.strftime("%Y%m")+"*.nc")[0])[var+["mu_cape"]].sel({"time":time.replace(minute=0)})
    
    lats = f.coords.get("lat").values
    lons = f.coords.get("lon").values
    x,y = np.meshgrid(lons,lats)
    dist_km = (latlon_dist(lat, lon, y, x) )
    mask = get_mask(lons,lats)
    a,b = np.where( (dist_km <= r) & (mask == 1) )
    target_lons = xr.DataArray(lons[b],dims="points")
    target_lats = xr.DataArray(lats[a],dims="points")    
    f_slice = (f.sel(lon=target_lons, lat=target_lats))
    
    if plot:
        plt.figure()
        ax=plt.axes(projection=ccrs.PlateCarree())
        temp = np.where((dist_km <= r) & (mask == 1), f.values, np.nan)
        c=ax.pcolormesh(x,y,temp,vmin=vmin,vmax=vmax)
        plt.colorbar(c)
        ax.coastlines()
    
    #Return the value of the point with the highest absolute value
    #print(f_slice)
    return pd.DataFrame([f_slice[v].values[np.abs(f_slice[v]).argmax()] for v in var], index=var),\
            [f_slice.lat.values[np.abs(f_slice["mu_cape"]).argmax()],\
             f_slice.lon.values[np.abs(f_slice["mu_cape"]).argmax()] ]

def latlon_dist(lat, lon, lats, lons):

        #Calculate great circle distance (Harversine) between a lat lon point (lat, lon) and a list of lat lon
        # points (lats, lons)
                        
        R = 6373.0
                        
        lat1 = np.deg2rad(lat)
        lon1 = np.deg2rad(lon)
        lat2 = np.deg2rad(lats)
        lon2 = np.deg2rad(lons)
                
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        return (R * c)

def get_mask(lon,lat,thresh=0.5):
    #Return lsm for a given domain (with lats=lat and lons=lon)
    lsm,nat_lon,nat_lat = get_lsm()
    lon_ind = np.where((nat_lon >= lon[0]) & (nat_lon <= lon[-1]))[0]
    lat_ind = np.where((nat_lat >= lat[-1]) & (nat_lat <= lat[0]))[0]
    lsm_domain = lsm[(lat_ind[0]):(lat_ind[-1]+1),(lon_ind[0]):(lon_ind[-1]+1)]
    lsm_domain = np.where(lsm_domain > thresh, 1, 0)

    return lsm_domain

def get_lsm():
    #Load the ERA5 land-sea fraction
    lsm_file = nc.Dataset("/g/data/rt52/era5/single-levels/reanalysis/lsm/1979/lsm_era5_oper_sfc_19790101-19790131.nc")
    lsm = np.squeeze(lsm_file.variables["lsm"][0])
    lsm_lon = np.squeeze(lsm_file.variables["longitude"][:])
    lsm_lat = np.squeeze(lsm_file.variables["latitude"][:])
    lsm_file.close()
    return [lsm,lsm_lon,lsm_lat]

def plot_prof(prof, ax, ax2, gs, hodo_size=1.7):
        pb_plot=1050
        pt_plot=100
        dp_plot=10
        plevs_plot = np.arange(pb_plot,pt_plot-1,-dp_plot)

        # Set up the figure in matplotlib.
        ax.grid(True)
        plt.grid(True)      
        
        # Plot the background variables
        presvals = np.arange(1000, 0, -10)
        ax.semilogy(prof.vtmp[~prof.vtmp.mask], prof.pres[~prof.vtmp.mask], 'r', lw=2)
        #ax.semilogy(prof.tmpc[~prof.tmpc.mask], prof.pres[~prof.tmpc.mask], 'r', lw=2)
        ax.semilogy(prof.dwpc[~prof.dwpc.mask], prof.pres[~prof.dwpc.mask], 'g', lw=2)

        # Plot the parcel trace, but this may fail.  If it does so, inform the user.
        sfcpcl = params.parcelx( prof, flag=1 )
        mupcl = params.parcelx( prof, flag=3 )
        mlpcl = params.parcelx( prof, flag=4 )
        ax.semilogy(sfcpcl.ttrace, sfcpcl.ptrace, 'k:')
        ax.semilogy(mlpcl.ttrace, mlpcl.ptrace, color='tab:blue', ls=":")
        ax.semilogy(mupcl.ttrace, mupcl.ptrace, color='tab:red', ls=":")

        # Disables the log-formatting that comes with semilogy
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.set_yticks(np.linspace(100,1000,10))
        ax.set_ylim(1050,100)
        [ax.text(-0.6, p, str(round(h/1000,1)) + " km", transform=ax.get_yaxis_transform(), va="center", size="x-large") 
             for h, p in zip(prof.hght[prof.pres >= 100][3:-1:2], prof.pres[prof.pres >= 100][3:-1:2])]

        # Plot the hodograph data.
        inset_axes = inset(ax,width=hodo_size, # width = 30% of parent_bbox
                                            height=hodo_size, # height : 1 inch
                                            loc=1)
        inset_axes.get_xaxis().set_visible(False)
        inset_axes.get_yaxis().set_visible(False)

        # Draw the range rings around the hodograph.
        for i in range(10,90,10):
            circle = Circle((0,0),i,color='k',alpha=.3, fill=False)
            if i % 10 == 0 and i <= 50:
                inset_axes.text(-i,2,str(i), fontsize=8, horizontalalignment='center')
            inset_axes.add_artist(circle)
        inset_axes.set_xlim(-60,60)
        inset_axes.set_ylim(-60,60)
        inset_axes.axhline(y=0, color='k')
        inset_axes.axvline(x=0, color='k')
        #srwind = tab.params.bunkers_storm_motion(prof)        
        skew.plotHodo(inset_axes, prof.hght, (prof.u), (prof.v), color='r')
        inset_axes.set_xlim([-50,60])
        inset_axes.set_ylim([-50,60])        

        # Draw the wind barbs axis and everything that comes with it.
        ax.set_xlim(-40,50)
        ax.set_xticks(np.arange(-40,60,10))
        skew.plot_wind_axes(ax2)
        skew.plot_wind_barbs(ax2, prof.pres, prof.u, prof.v)
        srwind = params.bunkers_storm_motion(prof)
        gs.update(left=0.05, bottom=0.05, top=0.95, right=1, wspace=0.025)       

        sfc = prof.pres[prof.sfc]
        p6km = interp.pres(prof, interp.to_msl(prof, 6000.))
        sfc_6km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p6km)
        s06 = utils.comp2vec(sfc_6km_shear[0], sfc_6km_shear[1])[1] * 0.514444
        print("MUCAPE: ",mupcl.bplus, "MLCAPE: ",mlpcl.bplus, "SBCAPE: ",sfcpcl.bplus, "S06: ",s06, "DCAPE",params.dcape(prof)[0])

def rotate_wind(prof, p_int):
    
    #From a profile (of u and v winds), rotate the wind profile so that Umean06 is westerly.
    
    u = interp.components(prof,p_int)[0].filled(np.nan)
    v = interp.components(prof,p_int)[1].filled(np.nan)

    x2,y2 = [1,0]
    x1,y1 = winds.mean_wind(prof,pbot=prof.pres.max(),ptop=interp.pres(prof,6000))
    
    angle = np.arctan2(x1*y2-y1*x2,x1*x2+y1*y2)

    u_prime = u*np.cos(angle) - v*np.sin(angle)
    v_prime = u*np.sin(angle) + v*np.cos(angle)

    ub = x2 / (np.sqrt(x2**2 + y2**2)) *1
    vb = y2 / (np.sqrt(x2**2 + y2**2)) *1    
    
    return u_prime, v_prime

def plot_prof_simple(prof, ax, ax2, hodo_size=1.7):
        pb_plot=1050
        pt_plot=100
        dp_plot=10
        plevs_plot = np.arange(pb_plot,pt_plot-1,-dp_plot)
        
        # Plot the background variables
        presvals = np.arange(1000, 0, -10)
        ax.semilogy(prof.vtmp[~prof.vtmp.mask], prof.pres[~prof.vtmp.mask], 'r', lw=2)
        ax.semilogy(prof.dwpc[~prof.dwpc.mask], prof.pres[~prof.dwpc.mask], 'g', lw=2)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)        

        # Plot the parcel trace, but this may fail.  If it does so, inform the user.
        mupcl = params.parcelx( prof, flag=3 )
        ax.semilogy(mupcl.ttrace, mupcl.ptrace, color='k')

        # Disables the log-formatting that comes with semilogy
        ax.yaxis.set_major_formatter(ScalarFormatter())
        ax.set_yticks(np.linspace(100,1000,10))
        ax.set_ylim(1050,100)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)      

        # Draw the wind barbs axis and everything that comes with it.
        ax.set_xlim(-40,50)
        ax.set_xticks(np.arange(-40,60,10))
        skew.plot_wind_axes(ax2)
        barb_pres = np.logspace(3,2,10)
        barb_u, barb_v = interp.components(prof, barb_pres)
        print(barb_pres, barb_u, barb_v)
        skew.plot_wind_barbs(ax2, barb_pres, barb_u, barb_v)
        srwind = params.bunkers_storm_motion(prof)
        
def load_scws(rid):
    print("loading "+rid+"...")
    df1 = pd.read_csv("/g/data/eg3/ab4502/ExtremeWind/points/"+rid+"_scw_envs_df.csv")

    df1["cluster_new"] = df1.cluster.map({1:2,2:1,0:0})
    df1 = df1.set_index(pd.DatetimeIndex(df1.dt_utc))
    df1["month"] = df1.index.month
    df1["hour"] = df1.index.hour
    df1["year"] = df1.index.year
    df1["aspect_ratio"] = df1.major_axis_length / df1.minor_axis_length    
    
    df1["rid"] = rid
    
    return df1        

In [121]:
df = pd.read_csv("/g/data/eg3/ab4502/figs/ExtremeWind/case_studies/cluster_input_era5.csv").drop(columns=["Unnamed: 0"])
var=np.array(df.columns)
scaler = MinMaxScaler()
df_norm = scaler.fit_transform(df)
mod=kmeans(n_clusters=3, verbose=0, random_state=0)
mod_fit=mod.fit(df_norm)

In [123]:
melb_scw  = load_scws("2")
bris_scw  = load_scws("66")
namoi_scw  = load_scws("69")
perth_scw  = load_scws("70")
syd_scw = load_scws("71")
scw = pd.concat([melb_scw,bris_scw,namoi_scw,perth_scw,syd_scw],axis=0)

loading 2...
loading 66...
loading 69...
loading 70...
loading 71...


In [161]:
closest1 = scw.iloc[np.argmin(mod_fit.transform(scaler.transform(scw[var]))[:,0])]#[["cluster_new","s06","qmean01","lr13","Umean06","mu_cape"]]
closest2 = scw.iloc[np.argmin(mod_fit.transform(scaler.transform(scw[var]))[:,2])]
closest3 = scw.iloc[np.argmin(mod_fit.transform(scaler.transform(scw[var]))[:,1])]
print(closest1[["rid","stn_id","cluster_new","s06","qmean01","lr13","Umean06","mu_cape"]], "\n\n",
      closest2[["rid","stn_id","cluster_new","s06","qmean01","lr13","Umean06","mu_cape"]], "\n\n",
      closest3[["rid","stn_id","cluster_new","s06","qmean01","lr13","Umean06","mu_cape"]], "\n\n")

rid                    2
stn_id             89002
cluster_new            0
s06            25.085938
qmean01         7.996094
lr13            6.889648
Umean06        23.601562
mu_cape         295.8125
Name: 2020-05-19 14:27:00, dtype: object 

 rid                   69
stn_id             61392
cluster_new            1
s06            27.328125
qmean01        10.179688
lr13            9.347656
Umean06        13.671875
mu_cape           1470.0
Name: 2017-02-18 09:07:00, dtype: object 

 rid                   66
stn_id             40842
cluster_new            2
s06            19.585938
qmean01        13.868164
lr13            7.155273
Umean06        12.273438
mu_cape        1478.1875
Name: 2020-10-25 04:43:00, dtype: object 




In [163]:


        

rid_info = {89002:{"lat":-37.5127,"lon":143.7911},61392:{"lat":-31.7416,"lon":150.7937}, 40842:{"lat":-27.3917,"lon":153.1292}}
prof1, mu_pcl1 = plot_sounding(closest1["dt_utc"].index,rid_info[closest1["stn_id"]]["lat"],rid_info[closest1["stn_id"]]["lon"], plot=False)

AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
closest1["dt_utc"]